In [1]:
# Import the required modules and libraries
from datetime import datetime, date, timezone
import pandas as pd
import numpy as np
import os

In [2]:
# Read each dataset create a pandas dataframe 
 
health_boards_df = pd.read_csv('datasets/health_boards.csv')
patients_demographics_df = pd.read_csv('datasets/patients_demographics.csv')
postcodes_df = pd.read_csv('datasets/postcodes.csv')
S08000020_df = pd.read_csv('datasets/S08000020.csv')
S08000024_df = pd.read_json('datasets/S08000024.json')
S08000031_df = pd.read_csv('datasets/S08000031.csv')

In [3]:
# Rename columns in health_borads_df and postcodes_df so that they can be merged with the paatient_demographics_df

health_boards_df.rename(columns = {"HB":"hb_id", "HBName":"hb_name"}, inplace=True)
postcodes_df.rename(columns = {"Postcode":"postal_code", 
                               "PostcodeDistrict":"postcode_district", 
                               "PostcodeSector":"postcode_sector", 
                               "ScottishIndexOfMultipleDeprivation2020Rank":"simd_rank"}, 
                    inplace=True)

patients_demographics_df.rename(columns = {"sex":"sex_at_birth"}, inplace=True)

In [4]:
# Modify/Extend patients_demographics_df
# Merge patients_demographics_df, health_boards_df and postcodes_df
# patients_demographics_df will now contain hb and postcode details 

patients_demographics_df = patients_demographics_df.merge(health_boards_df[["hb_id", "hb_name"]], 
                                                          on = "hb_id", how = "left").merge(postcodes_df[["postal_code",
                                                                                                          "postcode_district", 
                                                                                                          "postcode_sector", 
                                                                                                          "simd_rank"]], 
                                                          on = "postal_code", how = "left")

### MERGE S08000031 with patients_demographics_df

In [5]:
#  create new column = 'user' on patients_demographics_df dataframe 
patients_demographics_df["user"] = None

# define function that will recreate the usercode found in the S08000031_df 

def getUserName(postcode, surname, firstname):
    """
    postcode = postocde
        replace white spaces from postoce 
    surname = family name
        replace white spaces and "-" 
    firstname = first name initial 

    """
    return (postcode.replace(" ","") + surname.replace(" ","").replace("-","") + firstname[0]).upper()

# apply function to the entire patients_demographics_df["user"] column where the bh_id is equal to S08000031 
# we only need this function on the S08000031 datset
    
patients_demographics_df["user"] = patients_demographics_df.apply(lambda row : getUserName(row["postal_code"], 
                                                                                           row["family_name"], 
                                                                                           row["given_name"]) 
                                                                 if row["hb_id"] == "S08000031" else None, axis = 1)

In [6]:
# rename S08000031_df columns to match the one's in the destination schema 

S08000031_df.rename(columns = {"test_id":"spm_id", 
                               "determination":"outcome", 
                               "date":"effective_date", 
                               "tested_with":"spm_test_type"}, 
                    inplace=True)

In [7]:
# Merge S08000031_df and patients_demographics_df on the newly created 'user' column 

S08000031_df = S08000031_df.merge(patients_demographics_df, on = "user", how = "left")

# drop 'user' column from both dataframes as the merged is done and the user column will not be used anymore  

S08000031_df = S08000031_df.drop("user", axis = 1)
patients_demographics_df = patients_demographics_df.drop("user", axis = 1)


### MERGE S08000020 with patients_demographics_df

In [8]:
# rename S08000020_df columns to match the one's in the destination schema 

S08000020_df.rename(columns = {"specimen":"spm_id",
                               "patient":"u_id",
                               "result":"outcome", 
                               "date_taken":"effective_date"}, inplace=True)


# Merge S08000020_df and patients_demographics_df 

S08000020_df = S08000020_df.merge(patients_demographics_df, on = "u_id", how = "left")


### MERGE S08000024 with patients_demographics_df

In [9]:
# Merge S08000024_df and patients_demographics_df 

S08000024_df = S08000024_df.merge(patients_demographics_df, on = "u_id", how = "left")

### Concatenate S08000020, S08000024 and S08000031 -> destination dataset

In [10]:
# concatanate all the 3 dataframes

destinationDf = pd.concat([S08000020_df, S08000024_df, S08000031_df], ignore_index = True)

In [11]:
# function to change date format to YYYY-MM-DD
def change_date(a):
    if not isinstance(a, float):
        try: date = datetime.fromisoformat(a[:-1]).astimezone(timezone.utc).date()
        except ValueError: 
            try: date = datetime.strptime(a, "%d/%b/%Y").date()
            except ValueError:
                try: date = datetime.strptime(a, "%Y-%m-%d").date() 
                except ValueError: 
                    try: date = datetime.strptime(a, "%d/%m/%Y").date()
                    except ValueError: 
                        try: date = datetime.strptime(a, "%d %b %Y").date()
                        except ValueError:
                            date = None
    else: date = None
    return date

# function to match every patient's age with the correct age group
def ageGroup(birthDate):
    if birthDate != None:
        today = date.today()
        age = today.year - birthDate.year -((today.month, today.day) < (birthDate.month, birthDate.day))
        if 0 <= age <= 11:
            return "0-11"
        elif 12 <= age <= 17:
            return "12-17"
        elif 18 <= age <= 39:
            return "18-39"
        elif 40 <= age <= 54:
            return "40-54"
        else:
            return '55+'
    else: return None
    
#  function to change the test outcome to int format 0, 1 or 2 

def getOutcome(value):
    try:
        if value.strip(" ") == "POS" or value[-1] == "P":
            return 2
        elif value.strip(" ") == "NEG" or value[-1] == "N":
            return 1
        elif value == None or value[-1] == "Z":
            return 0
    except:
        return value


In [12]:
# apply functions on the destinationDf columns 

destinationDf["effective_date"] = destinationDf.apply(lambda row : change_date(row["effective_date"]), axis = 1)

#  first applly lambda function to destinationDf['dob'] column 
destinationDf["dob"] = destinationDf.apply(lambda row : change_date(row["dob"]), axis = 1)
destinationDf['age_group'] = destinationDf.apply(lambda row : ageGroup(row['dob']), axis = 1)

In [13]:
# change test ouctome to desired format and data type

destinationDf['outcome'] = destinationDf.apply(lambda row : getOutcome(row['outcome']), axis = 1) 
destinationDf["outcome"] = destinationDf["outcome"].fillna(0)
destinationDf["outcome"] = destinationDf["outcome"].astype(np.int64)


In [14]:
# change specimen type to desired format and data type

destinationDf['spm_test_type'] = destinationDf['spm_test_type'].replace('Lateral Flow Device','lfd').replace('Polymerase Chain Reaction','pcr').fillna('na')

In [15]:
# substitue Nan values in simd_rank with 0 and change dtype to int64

destinationDf["simd_rank"] = (destinationDf["simd_rank"].fillna(0).astype(np.int64))


In [16]:
# drop unwated columns from destinationDf

destinationDf = destinationDf.drop(["dob","given_name", "family_name", "postal_code", "email","mobile_number"], axis = 1)

# rearrange columns in the destinationDf

destinationDf = destinationDf[['spm_id', 
                               'hb_id',
                               'hb_name', 
                               'u_id', 
                               'sex_at_birth',
                               'postcode_district', 
                               'postcode_sector',
                               'simd_rank',
                               'outcome',
                               'effective_date',
                               'spm_test_type']]


In [17]:
destinationDf

,spm_id,hb_id,hb_name,u_id,sex_at_birth,postcode_district,postcode_sector,simd_rank,outcome,effective_date,spm_test_type
0,1FTEW1CM,S08000020,NHS Grampian,1G4GC5ER0DF656630,F,AB12,AB12 9,3888,2,2022-01-01,na
1,WAUNF78P,S08000020,NHS Grampian,3VW8S7ATXFM709021,F,AB51,AB51 0,5886,1,2022-01-06,na
2,WA1CFAFP,S08000020,NHS Grampian,1G6DJ1ED2B0519654,M,AB30,AB30 1,4664,1,2022-01-11,na
3,JTHBB1BA,S08000020,NHS Grampian,1G6YV36A695848747,M,AB16,AB16 5,2631,2,2022-01-12,na
4,1GYS3KEF,S08000020,NHS Grampian,5GAKRBED8BJ816443,F,AB34,AB34 5,4596,2,2022-01-11,na
...,...,...,...,...,...,...,...,...,...,...,...
2995,3c3c73a1-2bd9-4802-934a-347d9343a1db,S08000031,NHS Greater Glasgow and Clyde,1HGCR2E7XFA201770,M,G3,G3 7,6520,2,2022-01-03,lfd
2996,6c805718-549a-4cbf-b480-5e3327812df2,S08000031,NHS Greater Glasgow and Clyde,1N6AA0EK7FN600163,M,G83,G83 0,2527,2,2022-01-04,pcr
2997,c1af8d99-f0f3-4cb0-a61b-6c5b1cf1bcdf,S08000031,NHS Greater Glasgow and Clyde,1FTEW1E86AK512258,F,G69,G69 7,5520,2,2022-01-11,pcr
2998,1aacae12-a8bb-45fc-8e49-b19de7e1e138,S08000031,NHS Greater Glasgow and Clyde,JTDKN3DU6A0891048,M,PA6,PA6 7,5900,2,None,lfd


In [18]:
#  export destination dataframe as destination_dataset
#  if diectory to store files does not exist, create one

if not os.path.exists('solutions_datasets'):
                os.makedirs('solutions_datasets')   
                
destinationDf.to_csv('solutions_datasets/destination_dataset.csv', index=False)

In [19]:
destinationDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   spm_id             3000 non-null   object
 1   hb_id              2982 non-null   object
 2   hb_name            2982 non-null   object
 3   u_id               2999 non-null   object
 4   sex_at_birth       2982 non-null   object
 5   postcode_district  2982 non-null   object
 6   postcode_sector    2982 non-null   object
 7   simd_rank          3000 non-null   int64 
 8   outcome            3000 non-null   int64 
 9   effective_date     2999 non-null   object
 10  spm_test_type      3000 non-null   object
dtypes: int64(2), object(9)
memory usage: 257.9+ KB


###  Insights and observations 

In [20]:
destinationDf.loc[destinationDf['u_id'].isnull()]

,spm_id,hb_id,hb_name,u_id,sex_at_birth,postcode_district,postcode_sector,simd_rank,outcome,effective_date,spm_test_type
2984,d8b0441d-db1e-49b1-a3cd-eaee3d1ae2cc,NaN,NaN,NaN,NaN,NaN,NaN,0,2,2022-01-07,pcr


####  Observation:

User code in S08000031 dataset is missing the first name initial 

In [21]:
destinationDf.loc[destinationDf['effective_date'].isnull()]

,spm_id,hb_id,hb_name,u_id,sex_at_birth,postcode_district,postcode_sector,simd_rank,outcome,effective_date,spm_test_type
2998,1aacae12-a8bb-45fc-8e49-b19de7e1e138,S08000031,NHS Greater Glasgow and Clyde,JTDKN3DU6A0891048,M,PA6,PA6 7,5900,2,None,lfd


####  Observation:

Invalid effective date due to invalid date in S08000031 dataset - 2022-02-31 

In [22]:
destinationDf.loc[destinationDf['hb_id'].isnull()]

,spm_id,hb_id,hb_name,u_id,sex_at_birth,postcode_district,postcode_sector,simd_rank,outcome,effective_date,spm_test_type
1057,1Hj6D2XgPoQ4GmUa,NaN,NaN,WAUAF78E45A130356,NaN,NaN,NaN,0,1,2022-01-02,pcr
1078,14GGhJQT9y5Q1Zxo,NaN,NaN,WAUAF78E45A130356,NaN,NaN,NaN,0,1,2022-01-07,lfd
1157,1Ga2ubcRkstCDE6D,NaN,NaN,WAUAF78E45A130356,NaN,NaN,NaN,0,1,2022-01-04,lfd
1434,1LdWUs5Yj99jPktc,NaN,NaN,WAUAF78E45A130356,NaN,NaN,NaN,0,2,2022-01-02,pcr
1520,1Nz7EDUyJCuAr4dj,NaN,NaN,WAUAF78E45A130356,NaN,NaN,NaN,0,1,2022-01-03,lfd
1596,1AfqsnhQpzSics1n,NaN,NaN,WAUAF78E45A130356,NaN,NaN,NaN,0,2,2022-01-03,lfd
1674,1LQgcKKupxsve4r1,NaN,NaN,WAUAF78E45A130356,NaN,NaN,NaN,0,2,2022-01-08,pcr
1712,14f5bxWaVgV2Lsd4,NaN,NaN,WAUAF78E45A130356,NaN,NaN,NaN,0,2,2022-01-03,lfd
1718,14VLaRYerJtfGfgL,NaN,NaN,WAUAF78E45A130356,NaN,NaN,NaN,0,2,2022-01-05,lfd
1724,15wymaPp8XjfvnpA,NaN,NaN,WAUAF78E45A130356,NaN,NaN,NaN,0,2,2022-01-04,lfd


####  Observation:

WAUAF78E45A130356 - u_id is not registered in the patients demographics dataframe 
####
1GD999CG5EF949071 - u_id is not registered in the patients demographics dataframe 